In [ ]:
import pandas as pd
import os
import folium
import win32com.client
from geopy.distance import geodesic
from folium.plugins import AntPath
import googlemaps
import math

def haversine(coord1, coord2):
    R = 6371
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    return R * c * 1000


def distance_point_to_line(a, b, c):
    numerator = abs((b[1] - a[1]) * c[0] - (b[0] - a[0]) * c[1] + b[0] * a[1] - b[1] * a[0])
    denominator = math.sqrt((b[1] - a[1]) ** 2 + (b[0] - a[0]) ** 2)
    if denominator!=0:
        return numerator / denominator
    else:
        return 0

def converter_coordenadas(coord):
    try:
        coord = coord.split('.')[0]
    except AttributeError:
        pass
    return str(coord)[:3] + '.' + str(coord)[3:]


def calcular_distancia_total(locais):
    distancia_total = 0.0
    for i in range(len(locais) - 1):
        ponto1 = (locais[i]['latitude'], locais[i]['longitude'])
        ponto2 = (locais[i + 1]['latitude'], locais[i + 1]['longitude'])
        distancia = geodesic(ponto1, ponto2).kilometers
        distancia_total += distancia
    return distancia_total

dic_atual = os.getcwd()
df_linhas = pd.read_excel(dic_atual+'\\linhas.xlsx', sheet_name='Folha1')
linhas_ = df_linhas['linhas'].values
for l in linhas_:
    linha_veiculos = str(l)
    df_matriz = pd.read_excel(dic_atual+'\\Motivos.xlsx', sheet_name='Folha2')
    conf = df_matriz[df_matriz['LINHA'] == linha_veiculos]
    ids_viagens = conf['ID_VIAGEM'].values
    for matriz in ids_viagens:
        id_viagem = str(matriz)
        df_viagens = pd.read_excel(dic_atual+"\\Motivos.xlsx", sheet_name='Folha2')
        
        conf = df_viagens[df_viagens['ID_VIAGEM'] == id_viagem]
        line = str(conf['LINHA'].values[0])
        sentido = "PC"+str(conf['PC'].values[0])
        sentido2 = str(conf['SUBLINHA'].values[0])
        veiculo_desejado = "NV="+str(conf['VEICULO'].values[0])
        distancia_dol = 1000*float(conf['EXTENSAO_REFERENCIAL'].values[0])
        veiculo_form = veiculo_desejado.replace('NV=', '')
        names = str(conf['DATA'].values[0]).split("T")[0]
        names = f'{names.split("-")[2]}-{names.split("-")[1]}-{names.split("-")[0]}'
        Nome_planilha_sateCar = names+' '+veiculo_form+'.xls'
        Nome_planilha_tacom = names+'.csv'
        qroentrada = str(conf['SAIDA'].values[0]).split(":")[0]+":"+str(conf['SAIDA'].values[0]).split(":")[1]
        qrosaida = str(conf['CHEGADA'].values[0]).split(":")[0]+":"+str(conf['CHEGADA'].values[0]).split(":")[1]
        
        excel_path = dic_atual + '\\Revalidações\\Relatório.xlsx'
        excel = win32com.client.Dispatch('Excel.Application')
        excel.Visible = True
        workbook = excel.Workbooks.Open(excel_path)
        sheet = workbook.Sheets('Viagens')
        
        caminho_arquivo_sateCar = dic_atual + f'\\Relatorios\\{Nome_planilha_sateCar}'
        caminho_arquivo_tacom = dic_atual + f'\\Relatorios\\{Nome_planilha_tacom}'
        caminho_arquivo_it = dic_atual +"\\Rotas.xlsx"
        
        
        df_it = pd.read_excel(caminho_arquivo_it, sheet_name=f'{line}_{str(conf['PC'].values[0])}_{sentido2}')
        latitude_it = df_it['Latitude'].values
        longitude_it = df_it['Longitude'].values
        distancia_total = 0.0
        i=0
        for i in range(len(latitude_it) - 1):
            ponto1 = (latitude_it[i], longitude_it[i])
            ponto2 = (latitude_it[i + 1], longitude_it[i + 1])
            distancia = geodesic(ponto1, ponto2).kilometers
            distancia_total += distancia
        print(distancia_total)
        ajuste=float(distancia_dol)/float(distancia_total)
        ajuste=ajuste/1000
        print(ajuste)
        itinerario_maps=[]
        y=1
        x=0
        ver=0
        distancia_total=0.0
        for latitude in latitude_it:
            longitude = longitude_it[x]
            if x!=0:
                ponto2 = (latitude, longitude)
                ponto1 = (latitude_it[x-1], longitude_it[x-1])
                distancia = geodesic(ponto1, ponto2).meters
                distancia_total=distancia_total+(distancia*ajuste)
            itinerario_maps.append({
                'latitude': float(latitude),
                'longitude': float(longitude),
                'informacao': int(distancia_total)
            })
            y=y+1
            x += 1
        #DESVIO-------------------------------------------------------------------------------------------------------------------------------------------
        sem_desvio = 0
        try:
            df_it = pd.read_excel(caminho_arquivo_it, sheet_name=f'{line}_{str(conf['PC'].values[0])}_{sentido2}_D')
        except ValueError:
            sem_desvio = 1
        if sem_desvio == 0:
            latitude_it = df_it['Latitude'].values
            longitude_it = df_it['Longitude'].values
            distancia_total = 0.0
            i=0
            for i in range(len(latitude_it) - 1):
                ponto1 = (latitude_it[i], longitude_it[i])
                ponto2 = (latitude_it[i + 1], longitude_it[i + 1])
                distancia = geodesic(ponto1, ponto2).kilometers
                distancia_total += distancia
            print(distancia_total)
            ajuste_D=float(distancia_dol)/float(distancia_total)
            ajuste_D=ajuste_D/1000
            print(ajuste_D)
            itinerario_maps_D=[]
            y=1
            x=0
            ver=0
            distancia_total=0.0
            for latitude in latitude_it:
                longitude = longitude_it[x]
                if x!=0:
                    ponto2 = (latitude, longitude)
                    ponto1 = (latitude_it[x-1], longitude_it[x-1])
                    distancia = geodesic(ponto1, ponto2).meters
                    distancia_total=distancia_total+(distancia*ajuste)
                itinerario_maps_D.append({
                    'latitude': float(latitude),
                    'longitude': float(longitude),
                    'informacao': int(distancia_total)
                })
                y=y+1
                x += 1
        #DESVIO-------------------------------------------------------------------------------------------------------------------------------------------
        #SateCar------------------------------------------------------------------------------------------------------------------------------------------
        SateCar_V=1
        locaisSet = []
        satecar = 0
        try:
            df_lat = pd.read_excel(caminho_arquivo_sateCar, sheet_name='Sheet1')
        except FileNotFoundError:
            SateCar_V=0
        if SateCar_V!=0:
            data_hora = df_lat['Data/Hora'].values
            latitude_ = df_lat['Unnamed: 1'].values
            longitude_ = df_lat['Unnamed: 2'].values
            y=1
            x=0
            for z in latitude_:
                longitude = str(longitude_[x])
                latitude = str(z)
                longitude = longitude.replace(".","")
                latitude = latitude.replace(".","")
                hora = data_hora[x]
                hora = hora.split(" ")[1]
                horas = int(hora[0:2])
                minutos = int(hora[3:5])
                hora = hora[0:2] + ':' + hora[3:5] + ':' + hora[6:8]
                if horas < int(qroentrada[0:2]) or (horas == int(qroentrada[0:2]) and minutos < int(qroentrada[3:5])):
                    x += 1
                    continue
                if horas == int(qrosaida[0:2]) and minutos > int(qrosaida[3:5]):
                    break
                
                latitude = converter_coordenadas(latitude)
                longitude = converter_coordenadas(longitude)
                if str(latitude_)!="nan":
                    locaisSet.append({
                        'latitude': float(latitude),
                        'longitude': float(longitude),
                        'informacao': f'Local {y}'
                    })
                    y=y+1
                    x += 1
            satecar = y
        
        
        #TACOM------------------------------------------------------------------------------------------------------------------------------------------
        df = pd.read_csv(caminho_arquivo_tacom, encoding='latin1').values
        locais = []
        horario = []
        x=0
        y=1
        for h in df:
            if df[x][0].split(';')[3]==veiculo_desejado:
                longitude = df[x][0].split(';')[1].replace('LG=', '')
                latitude = df[x][0].split(';')[2].replace('LT=', '')
                longitude = longitude.replace(".","")
                latitude = latitude.replace(".","")
                hora = df[x][0].split(';')[4]
                horas = int(hora[11:13])
                minutos = int(hora[13:15])
                if horas < int(qroentrada[0:2]) or (horas == int(qroentrada[0:2]) and minutos < int(qroentrada[3:5])):
                    x += 1
                    continue
                if horas == int(qrosaida[0:2]) and minutos > int(qrosaida[3:5]):
                    x += 1
                    continue
                if horas > int(qrosaida[0:2]):
                    x += 1
                    continue
                hora = hora[11:13] + ':' + hora[13:15] + ':' + hora[15:17]
                latitude = converter_coordenadas(latitude)
                longitude = converter_coordenadas(longitude)
                horario.append(hora)
                locais.append({
                    'latitude': float(latitude),
                    'longitude': float(longitude),
                    'informacao': f'Local {y}'
                })
                y=y+1
            x += 1
        if y==1:
            tacom=0
        else:
            tacom = y
    
        
        caminho_arquivo = dic_atual + '/GTFS.xlsx'
        df_trips = pd.read_excel(caminho_arquivo, sheet_name='trips')
        df_stop_times = pd.read_excel(caminho_arquivo, sheet_name='stop_times')
        df_routes = pd.read_excel(caminho_arquivo, sheet_name='routes')
        
        linha = df_routes[df_routes['route_short_name'] == line]
        pc = linha['route_id'].values[0]
        
        linha = df_trips[df_trips['route_id'] == int(pc)]
        trips = linha['trip_id'].values
        shapes_id = linha['shape_id'].values
        
        for x in trips:
            trip=x
        for x in shapes_id:
            shape_id=x
        
        teste_pc = shape_id-shapes_id[0]
        linha = df_stop_times[df_stop_times['trip_id'] == trips[0]]
        latitude_pc2 = linha['Latitude'].values
        longitude_pc2 = linha['Longitude'].values
        travel_pc2 = linha['shape_dist_traveled'].values
        
        linha = df_stop_times[df_stop_times['trip_id'] == trip]
        latitude_pc1 = linha['Latitude'].values
        longitude_pc1 = linha['Longitude'].values
        travel_pc1 = linha['shape_dist_traveled'].values
        
        pc_separado=shape_id-shapes_id[0]
        
        
        if sentido=="PC1":
            x=0
            itinerariopc1 = []
            for pc in latitude_pc1:
                latitude = pc
                longitude = longitude_pc1[x]
                
                latitude = converter_coordenadas(latitude)
                longitude = converter_coordenadas(longitude)
                
                itinerariopc1.append({
                    'latitude': float(latitude),
                    'longitude': float(longitude),
                    'informacao': f'Metros {travel_pc1[x]}'
                })
                x += 1
        
        
        if sentido=="PC2":
            x=0
            itinerariopc2 = []
            for pc in latitude_pc2:
                latitude = pc
                longitude = longitude_pc2[x]
                
                latitude = converter_coordenadas(latitude)
                longitude = converter_coordenadas(longitude)
                
                itinerariopc2.append({
                    'latitude': float(latitude),
                    'longitude': float(longitude),
                    'informacao': f'Metros {travel_pc2[x]}'
                })
                x += 1
        
        if locais:
            mapa = folium.Map(location=[locais[0]['latitude'], locais[0]['longitude']], zoom_start=12)
        elif locaisSet:
            mapa = folium.Map(location=[locaisSet[0]['latitude'], locaisSet[0]['longitude']], zoom_start=12)
        else:
            x=2
            itinerario_it=0
            while True:
                print(sheet.Cells(x, 1))
                if sheet.Cells(x, 1).Value==id_viagem or sheet.Cells(x, 1).Value==None:
                    sheet.Cells(x, 1).Value = id_viagem
                    sheet.Cells(x, 2).Value = line
                    sheet.Cells(x, 3).Value = sentido
                    sheet.Cells(x, 4).Value = veiculo_form
                    sheet.Cells(x, 5).Value = distancia_dol
                    sheet.Cells(x, 6).Value = names
                    sheet.Cells(x, 7).Value = "Sem sinal nenhum de gps TACOM"
                    sheet.Cells(x, 8).Value = "Inválido"
                    break
                x=x+1
            print(f"ID: {id_viagem}, sem nenhum cadastro de coordenadas TACOM nem SateCar.")
            workbook.Save()
            continue
        #TACOM ACEITE------------------------------------------------------------------------------------------------------------------------------------------
        contTac=0
        if locais:
            itinerario_ret=itinerario_maps
            relatorio = []
            tamanho = len(itinerario_ret)-1
            cont=0
            for x in locais:
                C = (x['latitude'], x['longitude'])
                y = 0
                ver=1
                cont=cont+1
                distance_ant=500
                for local in itinerario_ret:
                    if y != tamanho:
                        A = (local['latitude'], local['longitude'])
                        B = (itinerario_ret[y+1]['latitude'], itinerario_ret[y+1]['longitude'])
                        distance_to_line = distance_point_to_line(A, B, C) * 111000
                        distance_to_A = haversine(A, C)
                        distance_to_B = haversine(B, C)
                        distance_to_AB = haversine(A, B)
                        if distance_to_line<distance_ant and (A[0]<C[0]<B[0] or A[0]>C[0]>B[0] or A[1]<C[1]<B[1] or A[1]>C[1]>B[1]):
                            distance_ant=distance_to_line
                            coord_A=A
                            coord_B=B
                            inf_m=float(local['informacao'])+float(distance_to_A)
                    y=y+1
                if distance_ant<20:
                    relatorio.append({
                        'coord': C,
                        'distancia': f'{distance_ant:.2f}',
                        'coord2': f'{coord_A}',
                        'coord3': f'{coord_B}',
                        'inf': f'{inf_m:.2f}',
                        'verificar': 'Válido'
                    })
                    ver=0
                if ver==1:
                    y=0
                    distance_ant = 500
                    for local in itinerario_ret:
                        if y != tamanho:
                            A = (local['latitude'], local['longitude'])
                            distance_to_A = haversine(A, C)
                            if distance_to_A<distance_ant:
                                distance_ant=distance_to_A
                                menor_distance=local['informacao']
                                coord_A=A
                        y=y+1
                    if distance_ant<=20:
                        relatorio.append({
                            'coord': C,
                            'distancia': f'{distance_ant:.2f}',
                            'coord2': f'{coord_A}',
                            'coord3': f'{coord_A}',
                            'inf': f'{float(menor_distance):.2f}',
                            'verificar': 'Válido'
                        })
                    else:
                        relatorio.append({
                            'coord': C,
                            'distancia': f'{distance_ant:.2f}',
                            'coord2': f'{coord_A}',
                            'coord3': f'{coord_A}',
                            'inf': f'{float(menor_distance):.2f}',
                            'verificar': 'Inválido'
                        })
            
            buffer_tacom_ret = folium.FeatureGroup(name='Buffer Itinerario-TACOM')
            pings_tacom_ret = folium.FeatureGroup(name="PINGS 1 em 1 km, 20 metros da via TACOM")
            metros=0
            x=0
            for local in relatorio:
                if contTac==int(distancia_dol/1000):
                    break
                if metros+1000>distancia_dol:
                    break
                if float(local['inf'])-metros>1000:
                    metros=metros+1000
                    x+=1
                    continue
                if float(local['inf'])>=metros:
                    if float(local['distancia'])<=20.0:
                        contTac+=1
                        folium.Circle(
                            location=local['coord'],
                            radius=20,
                            color='green',
                            fill=True,
                            fill_color='purple',
                            popup=f'{local['inf']}m'
                        ).add_to(buffer_tacom_ret)
                        folium.Marker(
                            location=local['coord'],
                            icon=folium.Icon(color='red'),
                            popup=f'{local['inf']}m'
                        ).add_to(pings_tacom_ret)
                        metros=metros+1000
                x+=1
            
            itinerario_it = folium.FeatureGroup(name='Itinerário')
            pontos = [(local['latitude'], local['longitude']) for local in itinerario_ret]
            folium.PolyLine(pontos, color="blue", weight=2.5, opacity=2).add_to(itinerario_it)
        
        #DESVIO ACEITE---------------------------------------------------------------------------------------------------------------------------------
        if locais:
            if sem_desvio == 0:   
                contTac_D=0
                if locais:
                    itinerario_ret_D=itinerario_maps_D
                    relatorio_D = []
                    tamanho = len(itinerario_ret_D)-1
                    cont=0
                    for x in locais:
                        C = (x['latitude'], x['longitude'])
                        y = 0
                        ver=1
                        cont=cont+1
                        distance_ant=500
                        for local in itinerario_ret_D:
                            if y != tamanho:
                                A = (local['latitude'], local['longitude'])
                                B = (itinerario_ret_D[y+1]['latitude'], itinerario_ret_D[y+1]['longitude'])
                                distance_to_line = distance_point_to_line(A, B, C) * 111000
                                distance_to_A = haversine(A, C)
                                distance_to_B = haversine(B, C)
                                distance_to_AB = haversine(A, B)
                                if distance_to_line<distance_ant and (A[0]<C[0]<B[0] or A[0]>C[0]>B[0] or A[1]<C[1]<B[1] or A[1]>C[1]>B[1]):
                                    distance_ant=distance_to_line
                                    coord_A=A
                                    coord_B=B
                                    inf_m=float(local['informacao'])+float(distance_to_A)
                            y=y+1
                        if distance_ant<20:
                            relatorio_D.append({
                                'coord': C,
                                'distancia': f'{distance_ant:.2f}',
                                'coord2': f'{coord_A}',
                                'coord3': f'{coord_B}',
                                'inf': f'{inf_m:.2f}',
                                'verificar': 'Válido'
                            })
                            ver=0
                        if ver==1:
                            y=0
                            distance_ant = 500
                            for local in itinerario_ret_D:
                                if y != tamanho:
                                    A = (local['latitude'], local['longitude'])
                                    distance_to_A = haversine(A, C)
                                    if distance_to_A<distance_ant:
                                        distance_ant=distance_to_A
                                        menor_distance=local['informacao']
                                        coord_A=A
                                y=y+1
                            if distance_ant<=20:
                                relatorio_D.append({
                                    'coord': C,
                                    'distancia': f'{distance_ant:.2f}',
                                    'coord2': f'{coord_A}',
                                    'coord3': f'{coord_A}',
                                    'inf': f'{float(menor_distance):.2f}',
                                    'verificar': 'Válido'
                                })
                            else:
                                relatorio_D.append({
                                    'coord': C,
                                    'distancia': f'{distance_ant:.2f}',
                                    'coord2': f'{coord_A}',
                                    'coord3': f'{coord_A}',
                                    'inf': f'{float(menor_distance):.2f}',
                                    'verificar': 'Inválido'
                                })
                    
                    buffer_tacom_ret_D = folium.FeatureGroup(name='Buffer Itinerario-TACOM')
                    pings_tacom_ret_D = folium.FeatureGroup(name="PINGS 1 em 1 km, 20 metros da via TACOM")
                    metros=0
                    x=0
                    for local in relatorio_D:
                        if contTac_D==int(distancia_dol/1000):
                            break
                        if metros+1000>distancia_dol:
                            break
                        if float(local['inf'])-metros>1000:
                            metros=metros+1000
                            x+=1
                            continue
                        if float(local['inf'])>=metros:
                            if float(local['distancia'])<=20.0:
                                contTac_D+=1
                                folium.Circle(
                                    location=local['coord'],
                                    radius=20,
                                    color='green',
                                    fill=True,
                                    fill_color='purple',
                                    popup=f'{local['inf']}m'
                                ).add_to(buffer_tacom_ret_D)
                                folium.Marker(
                                    location=local['coord'],
                                    icon=folium.Icon(color='red'),
                                    popup=f'{local['inf']}m'
                                ).add_to(pings_tacom_ret_D)
                                metros=metros+1000   
                        x+=1
                    
                    itinerario_it_D = folium.FeatureGroup(name='Itinerário')
                    pontos = [(local['latitude'], local['longitude']) for local in itinerario_ret_D]
                    folium.PolyLine(pontos, color="blue", weight=2.5, opacity=2).add_to(itinerario_it_D)
                print(f'Itinerario {contTac}')
                print(f'Itinerario Desvio {contTac_D}')
                if contTac>=contTac_D:
                    itinerario_it.add_to(mapa)
                    buffer_tacom_ret.add_to(mapa)
                    pings_tacom_ret.add_to(mapa)
                else:
                    itinerario_it_D.add_to(mapa)
                    buffer_tacom_ret_D.add_to(mapa)
                    pings_tacom_ret_D.add_to(mapa)
                    relatorio = relatorio_D
                    itinerario_ret = itinerario_ret_D
                    contTac = contTac_D
            else:
                itinerario_it.add_to(mapa)
                buffer_tacom_ret.add_to(mapa)
                pings_tacom_ret.add_to(mapa)
            
        #DESVIO ACEITE----------------------------------------------------------------------------------------------------------------------------------
        
        #SATECAR ACEITE------------------------------------------------------------------------------------------------------------------------------------------
        if locaisSet:
            itinerario_ret=itinerario_maps
            relatorioSet = []
            tamanho = len(itinerario_ret)-1
            cont=0
            for x in locaisSet:
                C = (x['latitude'], x['longitude'])
                y = 0
                ver=1
                cont=cont+1
                distance_ant=500
                for local in itinerario_ret:
                    if y != tamanho:
                        A = (local['latitude'], local['longitude'])
                        B = (itinerario_ret[y+1]['latitude'], itinerario_ret[y+1]['longitude'])
                        distance_to_line = distance_point_to_line(A, B, C) * 111000
                        distance_to_A = haversine(A, C)
                        distance_to_B = haversine(B, C)
                        distance_to_AB = haversine(A, B)
                        if distance_to_line<distance_ant and (A[0]<C[0]<B[0] or A[0]>C[0]>B[0] or A[1]<C[1]<B[1] or A[1]>C[1]>B[1]):
                            distance_ant=distance_to_line
                            coord_A=A
                            coord_B=B
                            inf_m=float(local['informacao'])+float(distance_to_A)
                    y=y+1
                if distance_ant<20:
                    relatorioSet.append({
                        'coord': C,
                        'distancia': f'{distance_ant:.2f}',
                        'coord2': f'{coord_A}',
                        'coord3': f'{coord_B}',
                        'inf': f'{inf_m:.2f}',
                        'verificar': 'Válido'
                    })
                    ver=0
                if ver==1:
                    y=0
                    distance_ant = 500
                    for local in itinerario_ret:
                        if y != tamanho:
                            A = (local['latitude'], local['longitude'])
                            distance_to_A = haversine(A, C)
                            if distance_to_A<distance_ant:
                                distance_ant=distance_to_A
                                menor_distance=local['informacao']
                                coord_A=A
                        y=y+1
                    if distance_ant<=20:
                        relatorioSet.append({
                            'coord': C,
                            'distancia': f'{distance_ant:.2f}',
                            'coord2': f'{coord_A}',
                            'coord3': f'{coord_A}',
                            'inf': f'{float(menor_distance):.2f}',
                            'verificar': 'Válido'
                        })
                    else:
                        relatorioSet.append({
                            'coord': C,
                            'distancia': f'{distance_ant:.2f}',
                            'coord2': f'{coord_A}',
                            'coord3': f'{coord_A}',
                            'inf': f'{float(menor_distance):.2f}',
                            'verificar': 'Inválido'
                        })
            buffer_SateCar_ret = folium.FeatureGroup(name='Buffer Itinerario-SateCar')
            pings_SateCar_ret = folium.FeatureGroup(name="PINGS 1 em 1 km, 20 metros da via SateCar")
            contSet=0
            metros=0
            x=0
            for local in relatorioSet:
                if contSet==int(distancia_dol/1000):
                    break
                if metros+1000>distancia_dol:
                    break
                if float(local['inf'])-metros>1000:
                    metros=metros+1000
                    x+=1
                    continue
                if float(local['inf'])>=metros:
                    if float(local['distancia'])<=20.0:
                        contSet+=1
                        print(contSet)
                        folium.Circle(
                            location=local['coord'],
                            radius=20,
                            color='green',
                            fill=True,
                            fill_color='purple',
                            popup=f'{local['inf']}m'
                        ).add_to(buffer_SateCar_ret)
                        folium.Marker(
                            location=local['coord'],
                            icon=folium.Icon(color='blue'),
                            popup=f'{local['inf']}m'
                        ).add_to(pings_SateCar_ret)
                        metros=metros+1000
                x+=1
    
        #DESVIO SateCar ACEITE---------------------------------------------------------------------------------------------------------------------------------
        if locaisSet:
            if sem_desvio == 0:
                contSet_D=0
                if locaisSet:
                    itinerario_ret_D=itinerario_maps_D
                    relatorio_D_Set = []
                    tamanho = len(itinerario_ret_D)-1
                    cont=0
                    for x in locaisSet:
                        C = (x['latitude'], x['longitude'])
                        y = 0
                        ver=1
                        cont=cont+1
                        distance_ant=500
                        for local in itinerario_ret_D:
                            if y != tamanho:
                                A = (local['latitude'], local['longitude'])
                                B = (itinerario_ret_D[y+1]['latitude'], itinerario_ret_D[y+1]['longitude'])
                                distance_to_line = distance_point_to_line(A, B, C) * 111000
                                distance_to_A = haversine(A, C)
                                distance_to_B = haversine(B, C)
                                distance_to_AB = haversine(A, B)
                                if distance_to_line<distance_ant and (A[0]<C[0]<B[0] or A[0]>C[0]>B[0] or A[1]<C[1]<B[1] or A[1]>C[1]>B[1]):
                                    distance_ant=distance_to_line
                                    coord_A=A
                                    coord_B=B
                                    inf_m=float(local['informacao'])+float(distance_to_A)
                            y=y+1
                        if distance_ant<20:
                            relatorio_D_Set.append({
                                'coord': C,
                                'distancia': f'{distance_ant:.2f}',
                                'coord2': f'{coord_A}',
                                'coord3': f'{coord_B}',
                                'inf': f'{inf_m:.2f}',
                                'verificar': 'Válido'
                            })
                            ver=0
                        if ver==1:
                            y=0
                            distance_ant = 500
                            for local in itinerario_ret_D:
                                if y != tamanho:
                                    A = (local['latitude'], local['longitude'])
                                    distance_to_A = haversine(A, C)
                                    if distance_to_A<distance_ant:
                                        distance_ant=distance_to_A
                                        menor_distance=local['informacao']
                                        coord_A=A
                                y=y+1
                            if distance_ant<=20:
                                relatorio_D_Set.append({
                                    'coord': C,
                                    'distancia': f'{distance_ant:.2f}',
                                    'coord2': f'{coord_A}',
                                    'coord3': f'{coord_A}',
                                    'inf': f'{float(menor_distance):.2f}',
                                    'verificar': 'Válido'
                                })
                            else:
                                relatorio_D_Set.append({
                                    'coord': C,
                                    'distancia': f'{distance_ant:.2f}',
                                    'coord2': f'{coord_A}',
                                    'coord3': f'{coord_A}',
                                    'inf': f'{float(menor_distance):.2f}',
                                    'verificar': 'Inválido'
                                })
                    
                    buffer_SetCar_ret_D_Set = folium.FeatureGroup(name='Buffer Itinerario-SateCar')
                    pings_SetCar_ret_D_Set = folium.FeatureGroup(name="PINGS 1 em 1 km, 20 metros da via SateCar")
                    metros=0
                    x=0
                    for local in relatorio_D_Set:
                        if contSet_D==int(distancia_dol/1000):
                            break
                        if metros+1000>distancia_dol:
                            break
                        if float(local['inf'])-metros>1000:
                            metros=metros+1000
                            x+=1
                            continue
                        if float(local['inf'])>=metros:
                            if float(local['distancia'])<=20.0:
                                contSet_D+=1
                                folium.Circle(
                                    location=local['coord'],
                                    radius=20,
                                    color='green',
                                    fill=True,
                                    fill_color='purple',
                                    popup=f'{local['inf']}m'
                                ).add_to(buffer_SetCar_ret_D_Set)
                                folium.Marker(
                                    location=local['coord'],
                                    icon=folium.Icon(color='blue'),
                                    popup=f'{local['inf']}m'
                                ).add_to(pings_SetCar_ret_D_Set)
                                metros=metros+1000   
                        x+=1
                print(f'Itinerario {contSet}')
                print(f'Itinerario Desvio {contSet_D}')
                if contSet>=contSet_D:
                    buffer_SateCar_ret.add_to(mapa)
                    pings_SateCar_ret.add_to(mapa)
                else:
                    buffer_SetCar_ret_D_Set.add_to(mapa)
                    pings_SetCar_ret_D_Set.add_to(mapa)
                    relatorioSet = relatorio_D_Set
                    itinerario_ret = itinerario_ret_D
                    contSet = contSet_D
            else:
                buffer_SateCar_ret.add_to(mapa)
                pings_SateCar_ret.add_to(mapa)
            
        #DESVIO SateCar ACEITE----------------------------------------------------------------------------------------------------------------------------------
        
        #Maps------------------------------------------------------------------------------------------------------------------------------------------
        grupo1 = folium.FeatureGroup(name='Pings Tacom')
        grupo2 = folium.FeatureGroup(name='Pings SateCar')
        bufferSet = folium.FeatureGroup(name='Buffer Satecar')
        itinerario_real = folium.FeatureGroup(name='Itinerário Pings')
        
        x=1
        for local in itinerario_ret:
            folium.Marker(
                location=[local['latitude'], local['longitude']],
                popup=f'Local {x}, \n{local['informacao']} metros',
                icon=folium.Icon(color='green')
            ).add_to(itinerario_real)
            x=x+1
        itinerario_real.add_to(mapa)
        if locais:
            for local in locais:
                folium.Marker(
                    location=[local['latitude'], local['longitude']],
                    popup=local['informacao'],
                    icon=folium.Icon(color='red')
                ).add_to(grupo1)
            grupo1.add_to(mapa)
            print(f"Pings realizados pela TACOM: {tacom}")
        if locaisSet:
            for local in locaisSet:
                folium.Marker(
                    location=[local['latitude'], local['longitude']],
                    popup=local['informacao'],
                    icon=folium.Icon(color='blue')
                ).add_to(grupo2)
            grupo2.add_to(mapa)
            print(f"Pings realizados pela SATECAR: {satecar}")
        
        
        if locaisSet:
            descricao_html = f"""
            <div style="position: absolute; 
                        top: 750px; right: 0px; width: 300px; height: 300px; 
                        background-color: white; z-index: 9999; 
                        font-size: 14px; padding: 10px; border: 1px solid black;">
                <h4>TACOM x SATECAR</h4>
                <p>Pings realizados pela TACOM: {tacom}</p>
                <p>Pings realizados pela SATECAR: {satecar}</p>
                <p>Pings realizados pela SATECAR em trechos de 1km: {contSet}</p>
                <p>Pings realizados pela TACOM em trechos de 1km: {contTac}</p>
            </div>
            """
        else:
            descricao_html = f"""
            <div style="position: absolute; 
                        top: 750px; right: 0px; width: 300px; height: 300px; 
                        background-color: white; z-index: 9999; 
                        font-size: 14px; padding: 10px; border: 1px solid black;">
                <h4>TACOM</h4>
                <p>Pings realizados pela TACOM: {tacom}</p>
                <p>Pings realizados pela TACOM em trechos de 1km: {contTac}</p>
            </div>
            """
        
        mapa.get_root().html.add_child(folium.Element(descricao_html))
        
        folium.LayerControl().add_to(mapa)
        
        folder_path = os.path.join(os.getcwd()+f'\\Revalidações\\{line}\\{veiculo_form}', id_viagem)
        
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        data_f = pd.DataFrame(columns=['Coordenadas TACOM', 'Distancia da via', 'Coordenadas Itinerário 1', 'Coordenadas Itinerário 2', 'Metragem', 'Verificar'])
        data_Set = pd.DataFrame(columns=['Coordenadas SetCar', 'Distancia da via', 'Coordenadas Itinerário 1', 'Coordenadas Itinerário 2', 'Metragem', 'Verificar'])
        
        mapa.save(folder_path+f'\\{id_viagem} SateCar x Tacom linha {line} {sentido} {veiculo_form}.html')
        
        #Save TACOM------------------------------------------------------------------------------------------------------------------------------------------
        if locais:
            for dado in relatorio:
                new_row = pd.DataFrame([{
                    'Coordenadas TACOM': dado['coord'],
                    'Distancia da via': dado['distancia'],
                    'Coordenadas Itinerário 1': dado['coord2'],
                    'Coordenadas Itinerário 2': dado['coord3'],
                    'Metragem': dado['inf'],
                    'Verificar': dado['verificar']
                }])
                data_f = pd.concat([data_f, new_row], ignore_index=True)
            data_f = pd.DataFrame(data_f)
            
            excel_file_path = os.path.join(folder_path, f'{id_viagem} TACOM linha {line} {sentido} {veiculo_form}.xlsx')
            
            data_f.to_excel(excel_file_path, index=False, sheet_name="TACOM")
        
        #Save SateCar------------------------------------------------------------------------------------------------------------------------------------------
        if locaisSet:
            for dado in relatorioSet:
                new_row = pd.DataFrame([{
                    'Coordenadas SetCar': dado['coord'],
                    'Distancia da via': dado['distancia'],
                    'Coordenadas Itinerário 1': dado['coord2'],
                    'Coordenadas Itinerário 2': dado['coord3'],
                    'Metragem': dado['inf'],
                    'Verificar': dado['verificar']
                }])
                data_Set = pd.concat([data_Set, new_row], ignore_index=True)
            data_Set = pd.DataFrame(data_Set)
            
            excel_file_path = os.path.join(folder_path, f'{id_viagem} SateCar linha {line} {sentido} {veiculo_form}.xlsx')
            
            data_Set.to_excel(excel_file_path, index=False, sheet_name="SateCar")
        #Save relatório--------------------------------------------------------------------------------------------------------------------------------------
        if locais or locaisSet:
            pings = float(distancia_dol/1000)
            x=2
            while True:
                if sheet.Cells(x, 1).Value==id_viagem or sheet.Cells(x, 1).Value==None:
                    sheet.Cells(x, 1).Value = id_viagem
                    sheet.Cells(x, 2).Value = line
                    sheet.Cells(x, 3).Value = sentido
                    sheet.Cells(x, 4).Value = veiculo_form
                    sheet.Cells(x, 5).Value = distancia_dol
                    sheet.Cells(x, 6).Value = names
                    if float(contTac)>=pings-(pings*0.1):
                        sheet.Cells(x, 7).Value = f"Mais de 90% da via validada, {contTac} pings"
                        sheet.Cells(x, 8).Value = "Validado"
                    else:
                        sheet.Cells(x, 7).Value = f"Menos de 90% da via validada, {contTac} pings"
                        sheet.Cells(x, 8).Value = "Inválido"
                    break
                x=x+1
            workbook.Save()
        
        print(f"Arquivo Excel criado e salvo em: {excel_file_path}")

47.69793971469788
1.0033137759460375
Pings realizados pela TACOM: 108
Arquivo Excel criado e salvo em: C:\Users\leandro.arruda\Desktop\Backup\Plot\Revalidações\S10\1047\ID1047202407221115\ID1047202407221115 TACOM linha S10 PC1 1047.xlsx
47.69793971469788
1.0033137759460375
Pings realizados pela TACOM: 121
Arquivo Excel criado e salvo em: C:\Users\leandro.arruda\Desktop\Backup\Plot\Revalidações\S10\1058\ID1058202407231200\ID1058202407231200 TACOM linha S10 PC1 1058.xlsx
47.69793971469788
1.0033137759460375
Pings realizados pela TACOM: 141
Arquivo Excel criado e salvo em: C:\Users\leandro.arruda\Desktop\Backup\Plot\Revalidações\S10\1058\ID1058202407240530\ID1058202407240530 TACOM linha S10 PC1 1058.xlsx
47.69793971469788
1.0033137759460375
Pings realizados pela TACOM: 197
Arquivo Excel criado e salvo em: C:\Users\leandro.arruda\Desktop\Backup\Plot\Revalidações\S10\1136\ID1136202407241630\ID1136202407241630 TACOM linha S10 PC1 1136.xlsx
47.69793971469788
1.0033137759460375
Pings realizado